In [1]:
import pandas as pd
import numpy as np
import math

import time

In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv')

In [3]:
def knn(train):
    ratings = train.pivot(index='movie_id', columns='user_id', values='rating')
    sim = ratings.corr(method='pearson')
    return { "sim": sim, "ratings": ratings}

In [4]:
# Implementando predição
def predict(model, u, i, k = 5):
    sim = model["sim"]
    ratings = model["ratings"]
    ratings_T = ratings.T
    if u not in sim or i not in ratings_T:
        return 0
    sim_users = sim[u][~sim[u].isna()].sort_values(ascending=False).index
    rated_users = ratings_T[i][ratings_T[i] > 0].index
    sim_k = np.intersect1d(sim_users, rated_users)
    top_k = [x for x in sim_users if x in sim_k][:k]
    sum_sim = 0
    dem = 0
    mean_u = ratings[u].mean(skipna = True)
    for v in top_k:
        dem += sim[u][v] * (ratings[v][i] - ratings[v].mean(skipna = True))
        sum_sim += sim[u][v]
    if sum_sim == 0:
        return 0
    return mean_u + (dem/sum_sim)

In [5]:
# Avaliando modelo
def rmse(model, test, k = 5):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i, k)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [6]:
def validation(model, data, train_split = 0.75, k = 5):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test, k))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [7]:
# rmse(train.values[:1000])

In [8]:
validation(knn, train, 0.75, 7)

Tempo de treinamento em segundos:  201.38351964950562
RMSE : 1.338803799829699
Tempo de predição em segundos:  2129.867509365082


## Execução 1 treino 90% k = 5
Tempo de treinamento em segundos:  149.07133269309998 <br />
RMSE : 3.803978433165255 <br />
Tempo de predição em segundos:  26.33492422103882 <br />

## Execução 2 treino 75% k = 5
Tempo de treinamento em segundos:  126.98204016685486 <br />
RMSE : 3.770118711499278 <br />
Tempo de predição em segundos:  76.77399587631226 <br />

## Execução 3 treino 75% k = 10
Tempo de treinamento em segundos:  110.51997590065002 <br />
RMSE : 3.7701172485677734 <br />
Tempo de predição em segundos:  71.33514952659607 <br />

## Execução 4 treino 75% k = 5 (misturando todos os usuários)
Tempo de treinamento em segundos:  211.7357943058014 <br />
RMSE : 1.0656656377523062 <br />
Tempo de predição em segundos:  2155.411029815674 <br />

## Execução 5 treino 75% k = 9 (misturando todos os usuários)
Tempo de treinamento em segundos:  263.8602533340454 <br />
RMSE : 1.67112944390123 <br />
Tempo de predição em segundos:  2692.204513311386 <br />

## Execução 6 treino 75% k = 7 (misturando todos os usuários)
Tempo de treinamento em segundos:  201.38351964950562 <br />
RMSE : 1.338803799829699 <br />
Tempo de predição em segundos:  2129.867509365082 <br />